# Customer Time Series EDA
Kaggle's "American Express - Default Prediction" Competition provides us with time series data from credit card customers. For each customer we have 188 data points for each of up to 13 points in time. Therefore most customers have `2444 = 188 * 13` data points. For each customer, we can plot the 188 line plots that show how each variable (for one specific customer) changes over time.

Furthermore we can plot time series for customers who default on their credit payment. This is `target = 1` and is shown in blue below. And we can plot customers who do not default. This is `target = 0` and shown in orange below. By looking at different customers and different variables, we can gain intuition about which variables predict default.

In this notebook we chose 33 variables that look interesting. For each variable, we plot 32 default customers' time series, and 32 non-default customers' time series. To the right of the line plots, we plot a histogram also. Discussion about this EDA is [here][4].

If you wish to view the other 155 features, then copy and edit this notebook and display whichever variables you want. From the competition description [here][1], we know there are 5 types of variables. And there are 188 variables in total.

* D_* = 96 Delinquency variables
* S_* = 21 Spend variables
* P_* = 3 Payment variables
* B_* = 40 Balance variables
* R_* = 28 Risk variables

When you are ready to build a time series model like an RNN or Transformer that takes advantage of what you see in these plots, check out my TF GRU starter notebook [here][2] and check out my TF Transformer starter notebook [here][3]

[1]: https://www.kaggle.com/competitions/amex-default-prediction/data
[2]: https://www.kaggle.com/cdeotte/tensorflow-gru-starter-0-787
[3]: https://www.kaggle.com/code/cdeotte/tensorflow-transformer-0-790
[4]: https://www.kaggle.com/competitions/amex-default-prediction/discussion/327761

# 客戶時間序列 EDA
Kaggle 的“美國運通 - 默認預測”競賽為我們提供了來自信用卡客戶的時間序列數據。對於每個客戶，我們有 188 個數據點，每個數據點最多 13 個時間點。因此，大多數客戶都有“2444 = 188 * 13”的數據點。對於每個客戶，我們可以繪製 188 個線圖，顯示每個變量（對於一個特定客戶）如何隨時間變化。

此外，我們可以為拖欠信用付款的客戶繪製時間序列。這是`target = 1`，如下圖藍色所示。我們可以繪製沒有違約的客戶。這是`target = 0`，下面用橙色顯示。通過查看不同的客戶和不同的變量，我們可以直觀地了解哪些變量可以預測違約。

在這個筆記本中，我們選擇了 33 個看起來很有趣的變量。對於每個變量，我們繪製了 32 個默認客戶的時間序列和 32 個非默認客戶的時間序列。在折線圖的右側，我們還繪製了一個直方圖。關於這個 EDA 的討論是 [這裡][4]。

如果您想查看其他 155 個功能，請複制並編輯此筆記本並顯示您想要的任何變量。從比賽描述 [這裡][1]，我們知道有 5 種類型的變量。總共有188個變量。

* D_* = 96 個拖欠變量
* S_* = 21 個支出變量
* P_* = 3 個付款變量
* B_* = 40 平衡變量
* R_* = 28 個風險變量

當您準備好利用您在這些圖中看到的內容構建時間序列模型（如 RNN 或 Transformer）時，請查看我的 TF GRU 入門筆記本 [此處][2] 並查看我的 TF Transformer 入門筆記本 [此處] [3]

[1]:https://www.kaggle.com/competitions/amex-default-prediction/data
[2]:https://www.kaggle.com/cdeotte/tensorflow-gru-starter-0-787
[3]:https://www.kaggle.com/code/cdeotte/tensorflow-transformer-0-790
[4]:https://www.kaggle.com/competitions/amex-default-prediction/discussion/327761

# Load Train Data

In [ ]:
# LOAD LIBRARIES
# 加載庫
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

# LOAD TRAIN DATA AND MERGE TARGETS ONTO FEATURES
# 加載訓練數據並將目標合併到特徵上
df = pd.read_csv('../input/amex-default-prediction/train_data.csv', nrows=100_000)
df.S_2 = pd.to_datetime(df.S_2)
df2 = pd.read_csv('../input/amex-default-prediction/train_labels.csv')
df = df.merge(df2,on='customer_ID',how='left')

In [ ]:
def plot_time_series(prefix='D', cols=None, display_ct=32):
    
    # DETERMINE WHICH COLUMNS TO PLOT
    # 確定要繪製的列
    if cols is not None and len(cols)==0: cols = None
    if cols is None:
        COLS = df.columns[2:-1]
        COLS = np.sort( [int(x[2:]) for x in COLS if x[0]==prefix] )
        COLS = [f'{prefix}_{x}' for x in COLS]
        print('#'*25)
        print(f'Plotting all {len(COLS)} columns with prefix {prefix}')
        print('#'*25)
    else:
        COLS = [f'{prefix}_{x}' for x in cols]
        print('#'*25)
        print(f'Plotting {len(COLS)} columns with prefix {prefix}')
        print('#'*25)

    # ITERATE COLUMNS
    # 迭代列
    for c in COLS:

        # CONVERT DATAFRAME INTO SERIES WITH COLUMN
        # 將數據幀轉換為帶有列的序列
        tmp = df[['customer_ID','S_2',c,'target']].copy()
        tmp2 = tmp.groupby(['customer_ID','target'])[['S_2',c]].agg(list).reset_index()
        tmp3 = tmp2.loc[tmp2.target==1]
        tmp4 = tmp2.loc[tmp2.target==0]

        # FORMAT PLOT
        # 格式化繪圖
        spec = gridspec.GridSpec(ncols=2, nrows=1,
                             width_ratios=[3, 1], wspace=0.1,
                             hspace=0.5, height_ratios=[1])
        fig = plt.figure(figsize=(20,10))
        ax0 = fig.add_subplot(spec[0])

        # PLOT 32 DEFAULT CUSTOMERS AND 32 NON-DEFAULT CUSTOMERS
        # 繪製 32 位默認客戶和 32 位非默認客戶
        t0 = []; t1 = []
        for k in range(display_ct):
            try:
                # PLOT DEFAULTING CUSTOMERS
                # 繪製默認客戶
                row = tmp3.iloc[k]
                ax0.plot(row.S_2,row[c],'-o',color='blue')
                t1 += row[c]
                # PLOT NON-DEFAULT CUSTOMERS
                # 繪製非默認客戶
                row = tmp4.iloc[k]
                ax0.plot(row.S_2,row[c],'-o',color='orange')
                t0 += row[c]
            except:
                pass
        plt.title(f'Feature {c} (Key: BLUE=DEFAULT, orange=no default)',size=18)

        # PLOT HISTOGRAMS
        # 繪製直方圖
        ax1 = fig.add_subplot(spec[1])
        try:
            # COMPUTE BINS
            # 計算箱
            t = t0+t1; mn = np.nanmin(t); mx = np.nanmax(t)
            if mx==mn:
                mx += 0.01; mn -= 0.01
            bins = np.arange(mn,mx+(mx-mn)/20,(mx-mn)/20 )
            # PLOT HISTOGRAMS
            # 繪製直方圖
            if np.sum(np.isnan(t1))!=len(t1):
                ax1.hist(t1,bins=bins,orientation="horizontal",alpha = 0.8,color='blue')
            if np.sum(np.isnan(t0))!=len(t0):
                ax1.hist(t0,bins=bins,orientation="horizontal",alpha = 0.8,color='orange')
        except:
            pass
        plt.show()

# Plot Delinquency Variables

# 繪製拖欠變量

In [ ]:
# LEAVE LIST BLANK TO PLOT ALL
# 將列表留空以繪製所有
plot_time_series('D',[39,41,47,45,46,48,54,59,61,62,75,96,105,112,124])

# Plot Spend Variables

# 繪製支出變量

In [ ]:
# LEAVE LIST BLANK TO PLOT ALL
# 將列表留空以繪製所有
plot_time_series('S',[3,7,19,23,26])

# Plot Payment Variables

# 繪製支付變量

In [ ]:
# LEAVE LIST BLANK TO PLOT ALL
# 將列表留空以繪製所有
plot_time_series('P',[2,3])

# Plot Balance Variables

# 繪製平衡變量

In [ ]:
# LEAVE LIST BLANK TO PLOT ALL
# 將列表留空以繪製所有
plot_time_series('B',[2,3,4,5,7,9,20])

# Plot Risk Variables

# 繪製風險變量

In [ ]:
# LEAVE LIST BLANK TO PLOT ALL
# 將列表留空以繪製所有
plot_time_series('R',[1,3,13,18])